# Imports

In [1]:
import plotly.express as px
import pandas as pd

In [2]:
from helpers import plot_correlation_matrix, create_correlation_matrix

# Read Data

This data set takes into account passing data from 2012 - 2022. This data is unfiltered.

In [49]:
df_roster = pd.read_pickle("../../interactive/df_roster_2012_2022.pkl")

In [48]:
df_seasonal = pd.read_pickle("../../interactive/df_seasonal_2012_2022.pkl")

In [6]:
df_pbp = pd.read_pickle("../../interactive/df_pbp_2012_2022.pkl")

In [7]:
df_pbp.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
0,1.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,74.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,95.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Creat Data Frame for Runs

In [11]:
df_runs = df_pbp.query("play_type == 'run' and season_type == 'REG'")

In [13]:
df_runs.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
2,53.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,74.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,165.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,256.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,ATL,away,KC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,359.0,2012_01_ATL_KC,2012090908,KC,ATL,REG,1,KC,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
for col in list(df_runs.columns):
    if "bro" in col:
        print(col)

# Creat DF for Rushing Stats by Season

In [40]:
df_rushing_yards_by_season = df_runs \
    .groupby(["rusher_id", "rusher", "season"]) \
    .agg({
        "rushing_yards": ["sum", "mean"],
        "rush_attempt": ["sum"],
        "fumble": ["sum"],
        "rush_touchdown": ["sum"],
        "tackled_for_loss": ["sum"],
        "epa": ["sum", "mean"],
        "success": ["sum"]
    })

In [41]:
df_rushing_yards_by_season.head()

rushing_yards      rush_attempt fumble  \
                                         sum mean          sum    sum   
rusher_id  rusher       season                                          
00-0007091 M.Hasselbeck 2012             0.0  0.0          1.0    1.0   
                        2014            -5.0 -5.0          1.0    0.0   
                        2015             4.0  4.0          1.0    0.0   
00-0010346 P.Manning    2012            -2.0 -1.0          2.0    0.0   
                        2013            -9.0 -0.9         10.0    5.0   

                               rush_touchdown tackled_for_loss        epa  \
                                          sum              sum        sum   
rusher_id  rusher       season                                              
00-0007091 M.Hasselbeck 2012              0.0              0.0  -1.360733   
                        2014              0.0              1.0  -1.280506   
                        2015              0.0              0.0  -0.035615   
00-0010346 P.Manning    2012              0.0              1.0  -0.817786   
                        2013              1.0              2.0 -18.549723   

                                         success  
                                    mean     sum  
rusher_id  rusher       season                    
00-0007091 M.Hasselbeck 2012   -1.360733     0.0  
                        2014   -1.280506     0.0  
                        2015   -0.035615     0.0  
00-0010346 P.Manning    2012   -0.408893     0.0  
                        2013   -1.854972     1.0

In [43]:
df_rushing_yards_by_season.columns = list(map("_".join, df_rushing_yards_by_season.columns))
df_rushing_yards_by_season.reset_index(inplace=True)

In [61]:
df_rushing_yards_by_season_cleaned = df_rushing_yards_by_season.dropna()

In [63]:
df_rushing_yards_by_season_cleaned.head()

,rusher_id,rusher,season,rushing_yards_sum,rushing_yards_mean,rush_attempt_sum,fumble_sum,rush_touchdown_sum,tackled_for_loss_sum,epa_sum,epa_mean,success_sum,games,fantasy_points,age
0,00-0007091,M.Hasselbeck,2012,0.0,0.0,1.0,1.0,0.0,0.0,-1.360733,-1.360733,0.0,8.0,76.48,36.0
1,00-0007091,M.Hasselbeck,2014,-5.0,-5.0,1.0,0.0,0.0,1.0,-1.280506,-1.280506,0.0,4.0,16.94,38.0
2,00-0007091,M.Hasselbeck,2015,4.0,4.0,1.0,0.0,0.0,0.0,-0.035615,-0.035615,0.0,8.0,91.10,39.0
3,00-0010346,P.Manning,2012,-2.0,-1.0,2.0,0.0,0.0,1.0,-0.817786,-0.408893,0.0,16.0,310.96,36.0
4,00-0010346,P.Manning,2013,-9.0,-0.9,10.0,5.0,1.0,2.0,-18.549723,-1.854972,1.0,16.0,409.98,37.0


In [70]:
for i, row in df_rushing_yards_by_season_cleaned.copy().iterrows():
    r_id = row["rusher_id"]
    season = row["season"]
    
    seasonal_row = df_seasonal.query(f"player_id == '{r_id}' & season == {season} & season_type == 'REG'")
    roster_row = df_roster.query(f"player_id == '{r_id}' & season == {season}")
    
    if seasonal_row.empty:
        print(r_id)
        print(season)
        print(seasonal_row)
    
    # from seasonal df
    df_rushing_yards_by_season_cleaned.at[i, "games"] = seasonal_row["games"].iloc[0]
    df_rushing_yards_by_season_cleaned.at[i, "fantasy_points"] = seasonal_row["fantasy_points"].iloc[0]
    
    # from roster df
    df_rushing_yards_by_season_cleaned.at[i, "age"] = roster_row["age"].iloc[0]
    df_rushing_yards_by_season_cleaned.at[i, "position"] = roster_row["position"].iloc[0]

In [71]:
df_rushing_yards_by_season_cleaned.head()

,rusher_id,rusher,season,rushing_yards_sum,rushing_yards_mean,rush_attempt_sum,fumble_sum,rush_touchdown_sum,tackled_for_loss_sum,epa_sum,epa_mean,success_sum,games,fantasy_points,age,position
0,00-0007091,M.Hasselbeck,2012,0.0,0.0,1.0,1.0,0.0,0.0,-1.360733,-1.360733,0.0,8.0,76.48,36.0,QB
1,00-0007091,M.Hasselbeck,2014,-5.0,-5.0,1.0,0.0,0.0,1.0,-1.280506,-1.280506,0.0,4.0,16.94,38.0,QB
2,00-0007091,M.Hasselbeck,2015,4.0,4.0,1.0,0.0,0.0,0.0,-0.035615,-0.035615,0.0,8.0,91.10,39.0,QB
3,00-0010346,P.Manning,2012,-2.0,-1.0,2.0,0.0,0.0,1.0,-0.817786,-0.408893,0.0,16.0,310.96,36.0,QB
4,00-0010346,P.Manning,2013,-9.0,-0.9,10.0,5.0,1.0,2.0,-18.549723,-1.854972,1.0,16.0,409.98,37.0,QB


# Create Lag Version of Data

In [72]:
df_now = df_rushing_yards_by_season.copy()
df_last = df_rushing_yards_by_season.copy()

In [75]:
rename_dict = {}
for col in list(df_last.columns[3:]):
    rename_dict[col] = f"{col}_last"

In [76]:
df_last.rename(columns=rename_dict, inplace=True)

In [77]:
df_last["season"] += 1

In [78]:
df_lag = df_now.merge(df_last, how='inner', on=['rusher_id', 'rusher', 'season'])

In [79]:
df_lag.head()

,rusher_id,rusher,season,rushing_yards_sum,rushing_yards_mean,rush_attempt_sum,fumble_sum,rush_touchdown_sum,tackled_for_loss_sum,epa_sum,...,rush_attempt_sum_last,fumble_sum_last,rush_touchdown_sum_last,tackled_for_loss_sum_last,epa_sum_last,epa_mean_last,success_sum_last,games_last,fantasy_points_last,age_last
0,00-0007091,M.Hasselbeck,2015,4.0,4.000000,1.0,0.0,0.0,0.0,-0.035615,...,1.0,0.0,0.0,1.0,-1.280506,-1.280506,0.0,4.0,16.94,38.0
1,00-0010346,P.Manning,2013,-9.0,-0.900000,10.0,5.0,1.0,2.0,-18.549723,...,2.0,0.0,0.0,1.0,-0.817786,-0.408893,0.0,16.0,310.96,36.0
2,00-0010346,P.Manning,2014,-13.0,-1.857143,7.0,3.0,0.0,2.0,-12.568753,...,10.0,5.0,1.0,2.0,-18.549723,-1.854972,1.0,16.0,409.98,37.0
3,00-0019596,T.Brady,2013,13.0,1.083333,12.0,3.0,0.0,2.0,-9.026926,...,13.0,1.0,3.0,2.0,4.275997,0.328923,9.0,16.0,344.28,35.0
4,00-0019596,T.Brady,2014,20.0,1.428571,14.0,1.0,0.0,0.0,-2.157746,...,12.0,3.0,0.0,2.0,-9.026926,-0.752244,5.0,16.0,251.52,36.0


# Data Manipulations

Keep a copy of our unfiltered data so it is untouched if we need to reset our exploration.

In [23]:
df_temp = df_lag_unfiltered.copy()

In [24]:
filter_query = ""
if filter_query:
    df_temp = df_temp.query(filter_query)

## Per Game Version

In [25]:
filter_cols = ["age", "yards_per_pass", "age_last", "yards_per_pass_last", "season", 
               "passer", "passer_id", "games", "games_last"
              ]

per_game_cols = []
for col in list(df_temp.columns):
    if col not in filter_cols:
        per_game_cols.append(col)

In [26]:
df_per_game = df_temp[["passer_id", "passer", "season", "age", "age_last"]].copy()
for col in per_game_cols:
    new_col = f"{col}_per_game"
    if "last" in col:
        df_per_game[new_col] = df_temp[col] / df_temp["games_last"]
    else:
        df_per_game[new_col] = df_temp[col] / df_temp["games"]

In [27]:
df_per_game.head()

,passer_id,passer,season,age,age_last,passing_yards_per_game,passing_attempts_per_game,total_epa_per_game,passing_touchdowns_per_game,completions_per_game,fantasy_points_per_game,passing_yards_last_per_game,passing_attempts_last_per_game,total_epa_last_per_game,passing_touchdowns_last_per_game,completions_last_per_game,fantasy_points_last_per_game
0,00-0007091,M.Hasselbeck,2013,37.0,36.0,43.333333,4.0000,1.269991,0.0000,2.333333,1.00000,170.875000,27.625,-0.419874,0.8750,17.250000,9.56000
1,00-0007091,M.Hasselbeck,2014,38.0,37.0,75.250000,11.0000,1.518184,0.5000,7.500000,4.23500,43.333333,4.000,1.269991,0.0000,2.333333,1.00000
2,00-0007091,M.Hasselbeck,2015,39.0,38.0,211.250000,32.0000,3.880949,1.1250,19.500000,11.38750,75.250000,11.000,1.518184,0.5000,7.500000,4.23500
3,00-0010346,P.Manning,2013,37.0,36.0,342.312500,41.1250,18.124193,3.4375,28.125000,25.62375,291.187500,36.250,11.813057,2.3125,25.000000,19.43500
4,00-0010346,P.Manning,2014,38.0,37.0,295.437500,37.1875,9.229780,2.4375,24.687500,19.54250,342.312500,41.125,18.124193,3.4375,28.125000,25.62375


In [28]:
df = df_per_game.copy()

# Generate Graphs

In [29]:
df.head()

,passer_id,passer,season,age,age_last,passing_yards_per_game,passing_attempts_per_game,total_epa_per_game,passing_touchdowns_per_game,completions_per_game,fantasy_points_per_game,passing_yards_last_per_game,passing_attempts_last_per_game,total_epa_last_per_game,passing_touchdowns_last_per_game,completions_last_per_game,fantasy_points_last_per_game
0,00-0007091,M.Hasselbeck,2013,37.0,36.0,43.333333,4.0000,1.269991,0.0000,2.333333,1.00000,170.875000,27.625,-0.419874,0.8750,17.250000,9.56000
1,00-0007091,M.Hasselbeck,2014,38.0,37.0,75.250000,11.0000,1.518184,0.5000,7.500000,4.23500,43.333333,4.000,1.269991,0.0000,2.333333,1.00000
2,00-0007091,M.Hasselbeck,2015,39.0,38.0,211.250000,32.0000,3.880949,1.1250,19.500000,11.38750,75.250000,11.000,1.518184,0.5000,7.500000,4.23500
3,00-0010346,P.Manning,2013,37.0,36.0,342.312500,41.1250,18.124193,3.4375,28.125000,25.62375,291.187500,36.250,11.813057,2.3125,25.000000,19.43500
4,00-0010346,P.Manning,2014,38.0,37.0,295.437500,37.1875,9.229780,2.4375,24.687500,19.54250,342.312500,41.125,18.124193,3.4375,28.125000,25.62375


In [31]:
df["passing_yards_per_game"].max()

371.2

In [32]:
list(df.columns)[5:]

['passing_yards_per_game',
 'passing_attempts_per_game',
 'total_epa_per_game',
 'passing_touchdowns_per_game',
 'completions_per_game',
 'fantasy_points_per_game',
 'passing_yards_last_per_game',
 'passing_attempts_last_per_game',
 'total_epa_last_per_game',
 'passing_touchdowns_last_per_game',
 'completions_last_per_game',
 'fantasy_points_last_per_game']

## Correlation Matrix

In [33]:
x = [
    'passing_yards_per_game',
    'passing_attempts_per_game',
    'total_epa_per_game',
    'passing_touchdowns_per_game',
    'completions_per_game',
    'fantasy_points_per_game'
]

In [34]:
y = [
    'passing_yards_last_per_game',
    'passing_attempts_last_per_game',
    'total_epa_last_per_game',
    'passing_touchdowns_last_per_game',
    'completions_last_per_game',
    'fantasy_points_last_per_game'
]

In [35]:
corr_mat, x, y = create_correlation_matrix(df, x, y)
plot_correlation_matrix(corr_mat, x, y)

# Filtered DFs

In [48]:
def filter_df(df, query, x, y, title=""):
    # create filtered df
    df_filter = df.copy()
    df_filter[["games", "games_last"]] = df_temp[["games", "games_last"]]
    
    if query:
        df_filter = df_filter.query(query)
    
    # create and show corr_mat
    corr_mat, x, y = create_correlation_matrix(df_filter, x, y)
    plot_correlation_matrix(corr_mat, x, y, title=f"per-game/{title}")
    
    # visualize highest correlation pairing
    series = corr_mat.max().iloc[0:]
    temp_arr = list(series)

    max_series_value = series.max()
    i = temp_arr.index(max_series_value)
    col = list(corr_mat.columns)[i]
    col_prev = col.replace("_per_game", "_last_per_game")
    
    fig = px.scatter(df_filter, x=col_prev, y=col,
                 hover_data=["passer", "season", "age"]
                )
    if title:
        fig_name = f"{title}-scatter-{col}"
        path = f"../../interactive/QB/stability-passing/per-game/{fig_name}.html"
        print(f"\nSave path: {fig_name}\n")
        fig.write_html(path)
    fig.show()
    
    return df_filter

## No Filter

In [49]:
filter_df(df, "", x, y, title="unfiltered")


Save path: unfiltered-scatter-passing_yards_per_game



,passer_id,passer,season,age,age_last,passing_yards_per_game,passing_attempts_per_game,total_epa_per_game,passing_touchdowns_per_game,completions_per_game,fantasy_points_per_game,passing_yards_last_per_game,passing_attempts_last_per_game,total_epa_last_per_game,passing_touchdowns_last_per_game,completions_last_per_game,fantasy_points_last_per_game,games,games_last
0,00-0007091,M.Hasselbeck,2013,37.0,36.0,43.333333,4.000000,1.269991,0.000000,2.333333,1.000000,170.875000,27.625000,-0.419874,0.875000,17.250000,9.560000,3.0,8.0
1,00-0007091,M.Hasselbeck,2014,38.0,37.0,75.250000,11.000000,1.518184,0.500000,7.500000,4.235000,43.333333,4.000000,1.269991,0.000000,2.333333,1.000000,4.0,3.0
2,00-0007091,M.Hasselbeck,2015,39.0,38.0,211.250000,32.000000,3.880949,1.125000,19.500000,11.387500,75.250000,11.000000,1.518184,0.500000,7.500000,4.235000,8.0,4.0
3,00-0010346,P.Manning,2013,37.0,36.0,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,291.187500,36.250000,11.813057,2.312500,25.000000,19.435000,16.0,16.0
4,00-0010346,P.Manning,2014,38.0,37.0,295.437500,37.187500,9.229780,2.437500,24.687500,19.542500,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,00-0036945,J.Fields,2022,23.0,22.0,149.466667,21.200000,0.334649,1.133333,12.800000,19.732000,155.833333,22.333333,0.922352,0.583333,13.250000,10.566667,15.0,12.0
598,00-0036971,T.Lawrence,2022,22.0,21.0,241.941176,34.235294,8.125579,1.470588,22.764706,17.389412,213.352941,35.176471,-0.338662,0.705882,21.058824,11.708235,17.0,17.0
599,00-0036972,M.Jones,2022,23.0,22.0,214.071429,31.428571,1.438546,1.000000,20.571429,12.148571,223.588235,30.588235,5.135010,1.294118,20.705882,13.231765,14.0,17.0
600,00-0037012,T.Lance,2022,22.0,21.0,97.000000,15.500000,-3.080925,0.000000,7.500000,6.230000,100.500000,11.833333,3.224023,0.833333,6.833333,10.820000,2.0,6.0


## 12 Games 12 Games Last

In [52]:
query = "games >= 12 and games_last >= 12"
filter_df(df, query, x, y, title="12-games")


Save path: 12-games-scatter-passing_attempts_per_game



,passer_id,passer,season,age,age_last,passing_yards_per_game,passing_attempts_per_game,total_epa_per_game,passing_touchdowns_per_game,completions_per_game,fantasy_points_per_game,passing_yards_last_per_game,passing_attempts_last_per_game,total_epa_last_per_game,passing_touchdowns_last_per_game,completions_last_per_game,fantasy_points_last_per_game,games,games_last
3,00-0010346,P.Manning,2013,37.0,36.0,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,291.187500,36.250000,11.813057,2.312500,25.000000,19.435000,16.0,16.0
4,00-0010346,P.Manning,2014,38.0,37.0,295.437500,37.187500,9.229780,2.437500,24.687500,19.542500,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,16.0,16.0
6,00-0019596,T.Brady,2013,36.0,35.0,271.437500,39.000000,8.530468,1.562500,23.750000,15.720000,301.687500,39.625000,13.721569,2.125000,25.062500,21.517500,16.0,16.0
7,00-0019596,T.Brady,2014,37.0,36.0,256.812500,36.375000,8.834653,2.062500,23.312500,17.378750,271.437500,39.000000,8.530468,1.562500,23.750000,15.720000,16.0,16.0
8,00-0019596,T.Brady,2015,38.0,37.0,298.125000,38.875000,11.165308,2.250000,25.125000,21.481250,256.812500,36.375000,8.834653,2.062500,23.312500,17.378750,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,00-0036442,J.Burrow,2022,25.0,24.0,279.687500,37.812500,9.712385,2.187500,25.875000,21.918750,288.187500,32.375000,11.223626,2.125000,22.875000,19.640000,16.0,16.0
596,00-0036898,D.Mills,2022,23.0,22.0,207.866667,31.866667,-0.805794,1.133333,19.466667,12.101333,204.923077,30.153846,0.334728,1.230769,20.230769,12.073846,15.0,13.0
597,00-0036945,J.Fields,2022,23.0,22.0,149.466667,21.200000,0.334649,1.133333,12.800000,19.732000,155.833333,22.333333,0.922352,0.583333,13.250000,10.566667,15.0,12.0
598,00-0036971,T.Lawrence,2022,22.0,21.0,241.941176,34.235294,8.125579,1.470588,22.764706,17.389412,213.352941,35.176471,-0.338662,0.705882,21.058824,11.708235,17.0,17.0


## 12 Games; 12 Game Last  |  150 passing yards per game; 150 passing yards last per game

In [53]:
query = "games >= 12 and games_last >= 12 and passing_yards_per_game >= 150 and passing_yards_last_per_game >= 150"
filter_df(df, query, x, y, "starters")


Save path: starters-scatter-completions_per_game



,passer_id,passer,season,age,age_last,passing_yards_per_game,passing_attempts_per_game,total_epa_per_game,passing_touchdowns_per_game,completions_per_game,fantasy_points_per_game,passing_yards_last_per_game,passing_attempts_last_per_game,total_epa_last_per_game,passing_touchdowns_last_per_game,completions_last_per_game,fantasy_points_last_per_game,games,games_last
3,00-0010346,P.Manning,2013,37.0,36.0,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,291.187500,36.250000,11.813057,2.312500,25.000000,19.435000,16.0,16.0
4,00-0010346,P.Manning,2014,38.0,37.0,295.437500,37.187500,9.229780,2.437500,24.687500,19.542500,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,16.0,16.0
6,00-0019596,T.Brady,2013,36.0,35.0,271.437500,39.000000,8.530468,1.562500,23.750000,15.720000,301.687500,39.625000,13.721569,2.125000,25.062500,21.517500,16.0,16.0
7,00-0019596,T.Brady,2014,37.0,36.0,256.812500,36.375000,8.834653,2.062500,23.312500,17.378750,271.437500,39.000000,8.530468,1.562500,23.750000,15.720000,16.0,16.0
8,00-0019596,T.Brady,2015,38.0,37.0,298.125000,38.875000,11.165308,2.250000,25.125000,21.481250,256.812500,36.375000,8.834653,2.062500,23.312500,17.378750,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,00-0036389,J.Hurts,2022,24.0,23.0,246.733333,30.666667,6.648603,1.466667,20.400000,25.202667,209.600000,28.666667,3.859287,1.066667,17.666667,20.810667,15.0,15.0
595,00-0036442,J.Burrow,2022,25.0,24.0,279.687500,37.812500,9.712385,2.187500,25.875000,21.918750,288.187500,32.375000,11.223626,2.125000,22.875000,19.640000,16.0,16.0
596,00-0036898,D.Mills,2022,23.0,22.0,207.866667,31.866667,-0.805794,1.133333,19.466667,12.101333,204.923077,30.153846,0.334728,1.230769,20.230769,12.073846,15.0,13.0
598,00-0036971,T.Lawrence,2022,22.0,21.0,241.941176,34.235294,8.125579,1.470588,22.764706,17.389412,213.352941,35.176471,-0.338662,0.705882,21.058824,11.708235,17.0,17.0


## Young

In [54]:
query = "games >= 12 and games_last >= 12 \
and passing_yards_per_game >= 150 and passing_yards_last_per_game >= 150 \
and age <= 30"
filter_df(df, query, x, y, "young")


Save path: young-scatter-completions_per_game



,passer_id,passer,season,age,age_last,passing_yards_per_game,passing_attempts_per_game,total_epa_per_game,passing_touchdowns_per_game,completions_per_game,fantasy_points_per_game,passing_yards_last_per_game,passing_attempts_last_per_game,total_epa_last_per_game,passing_touchdowns_last_per_game,completions_last_per_game,fantasy_points_last_per_game,games,games_last
77,00-0023436,A.Smith,2014,30.0,29.0,217.666667,30.933333,6.976485,1.200000,20.200000,14.666667,220.866667,33.600000,5.319487,1.533333,20.533333,16.881333,15.0,15.0
145,00-0026143,M.Ryan,2013,28.0,27.0,282.187500,40.625000,5.935121,1.625000,27.437500,15.506250,294.687500,38.375000,11.166431,1.937500,26.312500,19.053750,16.0,16.0
146,00-0026143,M.Ryan,2014,29.0,28.0,293.375000,38.875000,7.167229,1.750000,25.937500,17.766250,282.187500,40.625000,5.935121,1.625000,27.437500,15.506250,16.0,16.0
147,00-0026143,M.Ryan,2015,30.0,29.0,286.937500,38.375000,8.041343,1.312500,25.437500,14.621250,293.375000,38.875000,7.167229,1.750000,25.937500,17.766250,16.0,16.0
157,00-0026158,J.Flacco,2013,28.0,27.0,244.312500,38.250000,2.148504,1.187500,22.562500,12.973750,238.562500,33.187500,4.490612,1.375000,19.812500,14.680000,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,00-0036389,J.Hurts,2022,24.0,23.0,246.733333,30.666667,6.648603,1.466667,20.400000,25.202667,209.600000,28.666667,3.859287,1.066667,17.666667,20.810667,15.0,15.0
595,00-0036442,J.Burrow,2022,25.0,24.0,279.687500,37.812500,9.712385,2.187500,25.875000,21.918750,288.187500,32.375000,11.223626,2.125000,22.875000,19.640000,16.0,16.0
596,00-0036898,D.Mills,2022,23.0,22.0,207.866667,31.866667,-0.805794,1.133333,19.466667,12.101333,204.923077,30.153846,0.334728,1.230769,20.230769,12.073846,15.0,13.0
598,00-0036971,T.Lawrence,2022,22.0,21.0,241.941176,34.235294,8.125579,1.470588,22.764706,17.389412,213.352941,35.176471,-0.338662,0.705882,21.058824,11.708235,17.0,17.0


## Old

In [55]:
query = "games >= 12 and games_last >= 12 \
and passing_yards_per_game >= 150 and passing_yards_last_per_game >= 150 \
and age_last >= 30"
filter_df(df, query, x, y, "old")


Save path: old-scatter-completions_per_game



,passer_id,passer,season,age,age_last,passing_yards_per_game,passing_attempts_per_game,total_epa_per_game,passing_touchdowns_per_game,completions_per_game,fantasy_points_per_game,passing_yards_last_per_game,passing_attempts_last_per_game,total_epa_last_per_game,passing_touchdowns_last_per_game,completions_last_per_game,fantasy_points_last_per_game,games,games_last
3,00-0010346,P.Manning,2013,37.0,36.0,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,291.187500,36.250000,11.813057,2.312500,25.000000,19.435000,16.0,16.0
4,00-0010346,P.Manning,2014,38.0,37.0,295.437500,37.187500,9.229780,2.437500,24.687500,19.542500,342.312500,41.125000,18.124193,3.437500,28.125000,25.623750,16.0,16.0
6,00-0019596,T.Brady,2013,36.0,35.0,271.437500,39.000000,8.530468,1.562500,23.750000,15.720000,301.687500,39.625000,13.721569,2.125000,25.062500,21.517500,16.0,16.0
7,00-0019596,T.Brady,2014,37.0,36.0,256.812500,36.375000,8.834653,2.062500,23.312500,17.378750,271.437500,39.000000,8.530468,1.562500,23.750000,15.720000,16.0,16.0
8,00-0019596,T.Brady,2015,38.0,37.0,298.125000,38.875000,11.165308,2.250000,25.125000,21.481250,256.812500,36.375000,8.834653,2.062500,23.312500,17.378750,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,00-0029604,K.Cousins,2022,34.0,33.0,266.823529,37.588235,6.035784,1.705882,24.882353,17.151765,263.812500,34.750000,7.744782,2.062500,23.250000,18.771250,17.0,16.0
353,00-0029701,R.Tannehill,2020,32.0,31.0,238.687500,30.062500,10.301593,2.062500,19.687500,21.710000,228.500000,23.583333,9.881447,1.833333,16.750000,18.681667,16.0,12.0
354,00-0029701,R.Tannehill,2021,33.0,32.0,219.647059,31.176471,5.615993,1.235294,21.000000,16.021176,238.687500,30.062500,10.301593,2.062500,19.687500,21.710000,17.0,16.0
355,00-0029701,R.Tannehill,2022,34.0,33.0,211.333333,27.000000,5.960948,1.083333,17.666667,13.770000,219.647059,31.176471,5.615993,1.235294,21.000000,16.021176,12.0,17.0


## Exploration

In [ ]:
query = "games >= 12 and games_last >= 12 \
and passing_yards_per_game >= 200 and passing_yards_last_per_game >= 200"
filter_df(df, query, x, y)